In [13]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta, date
from collections import Counter
import matplotlib.ticker as plticker
import pickle, os, tldextract
pd.set_option('display.max_columns', None)
import nltk
import spacy
import locationtagger

In [15]:
def initialize_style_parameters(small = False):
  
    if small == False:
        font = {
                'family' : 'DejaVu Sans',
                'weight' : 'normal',
                'size'   : 22
                }
    else:
        font = {
                'family' : 'DejaVu Sans',
                'weight' : 'normal',
                'size'   : 15
                }

    plt.rc('font', **font)

In [16]:
def cdf(xlist, namelist, title, colors, x_title = "", xlog=False, filename = "", size = (15, 6)):
    
    initialize_style_parameters()
    
    fig, ax = plt.subplots(figsize=(size[0],size[1]))
    for x,n,c in zip(xlist,namelist,colors):
        sorteddiff = np.sort(x)
        p = 1. * np.arange(len(sorteddiff))/(len(sorteddiff) - 1)
        plt.plot(sorteddiff, p,label=n, color=c, linewidth=3)

    if(xlog==True):
        plt.xscale('symlog')
        
    plt.ylabel('cumulative distribution function')
    plt.tight_layout()
    plt.grid(linestyle='dotted',linewidth=1.5)
    ax.legend(facecolor='white', framealpha=1, frameon = True)
    plt.xlabel(x_title)
    if filename!='':
        plt.savefig(filename,bbox_inches="tight")
        
    ax.set_title(title)
    plt.show()

In [17]:
def boxplot(data, xlabel, ylabel, width = 0.3, style = "whitegrid", x = None, y = None, hue = None, colors = None, small = False, img_name = None):
    
    fig, ax = plt.subplots(figsize=(15,6))
    sns.set_style(style)

    if colors is not None:
        sns.set_palette(sns.color_palette(colors))

    sns.boxplot(ax = ax, data = data, width = width, x = x, y = y, hue = hue).set(
        xlabel = xlabel, 
        ylabel = ylabel,
        yscale = "log"
    )

    if img_name is not None:
        save_pdf_img(plt, img_name)

    plt.show()

In [18]:
US_PAGES_PATH = "page_list/US_PAGES_TO_USE_FOR_ANALYSIS_final.csv"
US_PAGES = pd.read_csv(US_PAGES_PATH,index_col=None)
#US_PAGES=US_PAGES[~US_PAGES["pageCategory"].isin(["MEDIA_CRITIC", "MEDIA_AGENCY"])]
US_PAGES

/tmp/ipykernel_2583022/1798229430.py:2: DtypeWarning: Columns (22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  US_PAGES = pd.read_csv(US_PAGES_PATH,index_col=None)


,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,page_id,page_name,firstPostDate,lastPostDate,firstWeekFollowers,currentFollowers,page_category,totalPosts,totalInteractions,avgWeeklyInteractions,meanWeeklyInteractions,activeWeeks,activeOct2020,activeJun2022,mainLanguage,currentTopAdminCountry,pctLinkPost,pctPhotoStatusPost,pctVideoPosts,topSharedDomains,mostSharedDomain,nbSharedDomains,avgPostsPerAcitveWeek,avgPostsPerWeek,page_verified,page_created_month,topAdminCountries,languageCodes,pageCategories,accountNames,nb_posts,topAdminCountry,languageCode,pageCategory,nbTopAdminCountries,nbLanguageCodes,nbPageCategories,nbAccountNames,adLib_NameChanged,adLib_AdminCountries,adLib_Organization,adLib_Address,adLib_TopAdminCountry,adLib_OrgCountry,adLib_nbAdmins,adLib_nbAdminCountries,is_dedicated,actual_pageid,page_id_2,profile_owner,first_detected_date,detected_2020,detected_2022,niche,history_file,first_detected_date_2020,first_detected_date_2022,number_of_times_detected_2020,number_of_times_detected_2022
0,0,0,0,0,0,0,159903787387343,South Whitehall Patch,2017-06-19,2022-06-20,0.0,1076.0,NEWS_SITE,2327,669.0,10.292308,8.0,65,False,True,en,US,95.659648,4.125483,0.214869,patch.com,patch.com,1,35.800000,8.915709,0,2010-12,['US'],['en'],['NEWS_SITE'],['South Whitehall Patch'],25,US,en,NEWS_SITE,1,1,1,1,0.0,United States (3),NaN,NaN,US,NaN,3.0,1.0,False,NaN,-1,-1,2022-06-02,False,True,True,../data/pages_posts_history_minhkha/1599037873...,2015-01-01,2022-06-02,0,17
1,1,1,1,1,1,1,136558953025181,Life & Culture by Tampa Bay Times,2017-06-19,2022-06-20,0.0,27209.0,NEWS_SITE,5704,55427.0,340.042945,307.0,163,False,True,en,US,98.98317,0.806452,0.210379,tampabay.com,tampabay.com,1,34.993865,21.854406,0,2010-06,['US'],['en'],['NEWS_SITE'],['Life & Culture by Tampa Bay Times'],23,US,en,NEWS_SITE,1,1,1,1,3.0,United States (7),NaN,NaN,US,NaN,7.0,1.0,True,1.365590e+14,-1,100064169466475,2022-06-02,False,True,True,../data/pages_posts_history_minhkha/1000641694...,2015-01-01,2022-06-02,0,17
2,2,2,2,2,2,2,869834083131145,Conservative Writers,2017-06-26,2022-06-20,0.0,958.0,MEDIA_NEWS_COMPANY,3406,108187.0,416.103846,47.0,260,True,True,en,US,78.038755,18.46741,3.493834,carolinajournal.com | blackcommunitynews.com |...,carolinajournal.com,48,13.100000,13.100000,0,2015-10,['US'],['en'],['MEDIA_NEWS_COMPANY'],['Conservative Writers'],22,US,en,MEDIA_NEWS_COMPANY,1,1,1,1,0.0,United States (7),Conservative Writers,"Brevard County, Florida, United States",US,US,7.0,1.0,False,NaN,-1,-1,2020-10-06,True,True,True,../data/pages_posts_history_minhkha/8698340831...,2020-10-06,2022-06-20,18,2
3,3,3,3,3,3,3,274928162876363,"Goldsboro, NC Local - News Break",2017-06-19,2022-06-20,0.0,3536.0,MEDIA_NEWS_COMPANY,7142,4035.0,23.735294,10.0,170,True,True,en,US,100.0,0.0,0.0,newsbreak.com | newsbreakapp.com,newsbreak.com,2,42.011765,27.363985,0,2016-07,['US'],['en'],['MEDIA_NEWS_COMPANY'],"['Goldsboro, NC Local - News Break']",73,US,en,MEDIA_NEWS_COMPANY,1,1,1,1,1.0,"United States (7), China (4)","Particle Media, Inc.",NaN,US,NaN,11.0,2.0,False,NaN,-1,-1,2020-10-14,True,True,True,../data/pages_posts_history_minhkha/2749281628...,2020-10-14,2022-06-02,15,24
4,4,4,4,4,4,4,350210358815966,"Lawrenceville, GA Local - News Break",2018-03-12,2022-06-20,0.0,1037.0,MEDIA_NEWS_COMPANY,7024,1287.0,7.895706,2.0,163,True,True,en,US,100.0,0.0,0.0,newsbreak.com,newsbreak.com,1,43.092025,31.497758,0,2018-03,['US'],['en'],['MEDIA_NEWS_COMPANY'],"['Lawrenceville, GA Local - News Break']",37,US,en,MEDIA_NEWS_COMPANY,1,1,1,1,1.0,"United States (8), China (4)","Particle Media, Inc.",NaN,US,NaN,12.0,2.0,False,NaN,-1,-1,2020-10-17,True,True,True,../data/pages_posts_history_minhkha/3502103588...,2020-10-17,2022-06-02,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

# 1. Extract the localisation

In [19]:
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.downloader.download('en_core_web_lg')
nltk.downloader.download('en_core_web_sm')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/slide/chouakis/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to
[nltk_data]     /home/slide/chouakis/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package treebank to
[nltk_data]     /home/slide/chouakis/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/slide/chouakis/nltk_data...
[nltk_data]   Unzipping taggers/maxent_treebank_pos_tagger.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /home/slide/chouakis/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Error loading en_core_web_lg: Package 'en_core_web_lg' not
[nltk_data]     found in index
[nltk_data] Error loading en_core_web_sm: Package 'en_core_web_sm' not
[nltk_data]     found in index
[nltk_data] Downloading package averaged_perceptro

True

In [20]:
location_per_page = dict()

In [21]:
for idx, page in US_PAGES.iterrows():

    
    if page["page_id"] in location_per_page:
        continue
        
    if str(page["history_file"]) == "nan":
        continue
        

    history = pd.read_csv(page["history_file"])
    
    about = history["page_description"].to_list()[0]
    name = history["page_name"].to_list()[0]

    if str(about) == "nan":
        about = "a"
    
    if str(type(name))=="<class 'int'>":
        name = "a"

    from_about = locationtagger.find_locations(text = about)
    from_name = locationtagger.find_locations(text = name)

    
    location_per_page[page["page_id"]] = {"name": {"country": from_name.countries, "states": from_name.regions, "cities":from_name.cities}, 
                                         "about": {"country": from_about.countries, "states": from_about.regions, "cities":from_about.cities}}
    
    
    
    

/tmp/ipykernel_2583022/1801998534.py:11: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  history = pd.read_csv(page["history_file"])
/tmp/ipykernel_2583022/1801998534.py:11: DtypeWarning: Columns (23,28) have mixed types. Specify dtype option on import or set low_memory=False.
  history = pd.read_csv(page["history_file"])
/tmp/ipykernel_2583022/1801998534.py:11: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  history = pd.read_csv(page["history_file"])
/tmp/ipykernel_2583022/1801998534.py:11: DtypeWarning: Columns (23,28,33,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  history = pd.read_csv(page["history_file"])
/tmp/ipykernel_2583022/1801998534.py:11: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  history = pd.read_csv(page["history_file"])
/tmp/ipykernel_2583022/1801998534.py:11: D

In [22]:
#CLEANING


IGNORE = ["Local", "Media", "America", "North", "University", "Us", "US", "Post", "Rock", "Magazine", "West", "Real", "Lake", "Host", "Eagle"]

IGNORE += ['Notre Dame', 'Kelly', 'Sarah', 'Morgan', 'Grant', 'Jesus', 'Canada', 'Perry', 'Anderson', 'Lee', 'Willson', 'Lee', 'Bill', 'Ryan', 'Baton Rouge', 'Boise', 'LaFayette', 'Thomas', 'Spanish', 'Smith', 'Paul', 'Block', 'Page', 'Part', 'Welcome', 'Power', 'Wayne', 'Faith', 'Mountain', 'Mark', 'Marion', 'Tyler', 'Scott', 'Home', 'English', 'Enterprise', 'Beach', 'Orange', 'Park', 'Freedom', 'Four', 'Eagle', 'Host', 'Real']

for page_id, locations in location_per_page.items():
    
    location_per_page[page_id]["name"]["cities"] = [item for item in locations["name"]["cities"] if item not in IGNORE]
    location_per_page[page_id]["about"]["cities"] = [item for item in locations["about"]["cities"] if item not in IGNORE]
        
        

In [36]:
local_news = []
regional_news = []
country_news = []

PAGES_WITH_GEOGRAPHICAL_CONTENT = []
CITIES = []
STATES = []

LOCAL_PAGES = []
GLOBAL_PAGES = []

for idx, row in US_PAGES.iterrows():
    
    
    
    if row["page_id"] not in location_per_page:
        continue
        
    location_for_page = location_per_page[row["page_id"]]
    
    if len([item for item in location_for_page["name"]["cities"] if item != "Media"]) > 0 or len([item for item in location_for_page["about"]["cities"] if item != "Media"]):
        local_news.append(row["page_id"])
        PAGES_WITH_GEOGRAPHICAL_CONTENT.append(row["page_id"])
        cities_for_page = [item for item in location_for_page["name"]["cities"] + location_for_page["about"]["cities"]]
        cities_for_page = list(set(cities_for_page))
        LOCAL_PAGES.append(row["page_id"])
        CITIES += cities_for_page
        
    elif len(location_for_page["name"]["states"]) > 0 or len(location_for_page["about"]["states"]) > 0:
        regional_news.append(row["page_id"])
        PAGES_WITH_GEOGRAPHICAL_CONTENT.append(row["page_id"])
        
        states_for_page = [item for item in location_for_page["name"]["states"] + location_for_page["about"]["states"]]
        states_for_page = list(set(states_for_page))
        LOCAL_PAGES.append(row["page_id"])

        STATES += states_for_page

    elif len(location_for_page["name"]["country"]) > 0 or len(location_for_page["about"]["country"]) > 0:
        country_news.append(row["page_id"])
        PAGES_WITH_GEOGRAPHICAL_CONTENT.append(row["page_id"])
        GLOBAL_PAGES.append(row["page_id"])


print("Number of city news pages:", len(local_news), "Fraction:", len(local_news)/US_PAGES.shape[0])
print()
print("Number of state news pages:", len(regional_news), "Fraction:", len(regional_news)/US_PAGES.shape[0])
print()
print("Number of LOCAL (state+city) news pages", len(LOCAL_PAGES), "Fraction", len(LOCAL_PAGES)/US_PAGES.shape[0])
print("---------------------------------------------------------------------------------------------------------")
print("Number of country news pages:", len(country_news), "Fraction:", len(country_news)/US_PAGES.shape[0])


Number of city news pages: 8636 Fraction: 0.521529077842865

Number of state news pages: 731 Fraction: 0.04414517784890392

Number of LOCAL (state+city) news pages 9367 Fraction 0.5656742556917689
---------------------------------------------------------------------------------------------------------
Number of country news pages: 85 Fraction: 0.005133160214988827


In [26]:
Counter(CITIES).most_common(5)

[('California', 189),
 ('Washington', 127),
 ('Florida', 121),
 ('New York', 105),
 ('Ohio', 100)]

In [31]:
print(len(LOCAL_PAGES))
US_PAGES[(US_PAGES["page_id"].isin(LOCAL_PAGES)) & (US_PAGES["niche"] == False)]

9367


,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,page_id,page_name,firstPostDate,lastPostDate,firstWeekFollowers,currentFollowers,page_category,totalPosts,totalInteractions,avgWeeklyInteractions,meanWeeklyInteractions,activeWeeks,activeOct2020,activeJun2022,mainLanguage,currentTopAdminCountry,pctLinkPost,pctPhotoStatusPost,pctVideoPosts,topSharedDomains,mostSharedDomain,nbSharedDomains,avgPostsPerAcitveWeek,avgPostsPerWeek,page_verified,page_created_month,topAdminCountries,languageCodes,pageCategories,accountNames,nb_posts,topAdminCountry,languageCode,pageCategory,nbTopAdminCountries,nbLanguageCodes,nbPageCategories,nbAccountNames,adLib_NameChanged,adLib_AdminCountries,adLib_Organization,adLib_Address,adLib_TopAdminCountry,adLib_OrgCountry,adLib_nbAdmins,adLib_nbAdminCountries,is_dedicated,actual_pageid,page_id_2,profile_owner,first_detected_date,detected_2020,detected_2022,niche,history_file,first_detected_date_2020,first_detected_date_2022,number_of_times_detected_2020,number_of_times_detected_2022
7,8,8,8,8,8,8,46649543331,7News - WHDH Boston,2017-06-19,2022-06-20,0.0,505337.0,MEDIA_NEWS_COMPANY,61962,24260981.0,92599.164122,86629.5,262,True,True,en,US,96.186372,2.890481,0.923146,whdh.com,whdh.com,1,236.496183,237.402299,1,2008-11,['US'],['en'],['MEDIA_NEWS_COMPANY'],['7News - WHDH Boston'],797,US,en,MEDIA_NEWS_COMPANY,1,1,1,1,1.0,United States (11),SUNBEAM TELEVISION CORPORATION,"North Bay Village, Miami-Dade County, Florida,...",US,US,11.0,1.0,True,4.664954e+10,-1,100064380353385,2020-10-01,True,True,False,../data/pages_posts_history_minhkha/1000643803...,2020-10-01,2022-06-02,30,30
22,29,29,29,29,29,29,114737265221655,WREG News Channel 3,2017-06-19,2022-06-20,0.0,605830.0,NEWS_SITE,67011,19715022.0,75248.175573,60963.5,262,True,True,en,US,82.001462,7.494292,10.504246,wreg.com,wreg.com,1,255.767176,256.747126,1,2010-04,['US'],['en'],['NEWS_SITE'],['WREG News Channel 3'],622,US,en,NEWS_SITE,1,2,1,1,1.0,United States (47) Philippines (2),"NEXSTAR MEDIA GROUP, INC.","Irving, Dallas County, Texas, United States",US,US,49.0,2.0,True,1.147373e+14,-1,100051129136792,2020-10-01,True,True,False,../data/pages_posts_history_minhkha/1000511291...,2020-10-01,2022-06-02,31,30
72,103,103,103,103,103,103,5953023255,Chicago Tribune,2017-06-19,2022-06-20,0.0,700715.0,NEWS_SITE,41688,13131795.0,51497.235294,46699.0,255,True,True,en,US,98.179332,0.724429,1.096239,chicagotribune.com | thedailymeal.com,chicagotribune.com,2,163.482353,159.724138,1,2007-11,['US'],['en'],['NEWS_SITE'],['Chicago Tribune'],415,US,en,NEWS_SITE,1,1,1,1,0.0,United States (40) Israel (17) India (2) Ukra...,TRIBUNE PUBLISHING COMPANY,"Chicago, Cook County, Illinois, United States",US,US,60.0,4.0,True,5.953023e+09,-1,100071184386238,2020-10-01,True,True,False,../data/pages_posts_history_minhkha/1000711843...,2020-10-01,2022-06-02,30,30
74,106,106,106,106,106,106,324955307584131,SiliconBeat,2017-06-19,2022-06-20,0.0,3721.0,NEWS_SITE,6584,11500.0,44.230769,33.0,260,True,True,en,US,99.893682,0.045565,0.060753,mercurynews.com | siliconbeat.com,mercurynews.com,2,25.323077,25.226054,0,2012-06,['US'],['en'],['NEWS_SITE'],['SiliconBeat'],56,US,en,NEWS_SITE,1,1,1,1,0.0,United States (10),NaN,NaN,US,NaN,10.0,1.0,False,NaN,-1,-1,2020-10-01,True,True,False,../data/pages_posts_history_minhkha/3249553075...,2020-10-01,2022-06-02,21,20
97,140,140,140,140,140,140,181613422554,The Darien Times - local news and sports of Da...,2017-06-19,2022-03-28,0.0,4281.0,TOPIC_NEWSPAPER,6489,105652.0,480.236364,377.0,220,True,False,en,US,92.772384,5.640314,1.587302,darientimes.com | nextdoor.com | twitter.com |...,darientimes.com,7,29.495455,26.060241,0,2009-11,['US'],['en'],['TOPIC_NEWSPAPER'],['The Darien Times - local news and sports of ...,12,US,en,TOPIC_NEWSPAPER,1,1,1,1,0.0,United States (12),Hearst,NaN,US,NaN,12.0,1.0,False,NaN,100054190366188,-1,2020-10-07,True,False,False,../data/pages_posts_history_minhkha/1816134225...,2020-10-07,2015-01-01,11,0
...,

In [32]:
print(len(GLOBAL_PAGES))
US_PAGES[(US_PAGES["page_id"].isin(GLOBAL_PAGES)) & (US_PAGES["niche"] == False)]

85


,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,page_id,page_name,firstPostDate,lastPostDate,firstWeekFollowers,currentFollowers,page_category,totalPosts,totalInteractions,avgWeeklyInteractions,meanWeeklyInteractions,activeWeeks,activeOct2020,activeJun2022,mainLanguage,currentTopAdminCountry,pctLinkPost,pctPhotoStatusPost,pctVideoPosts,topSharedDomains,mostSharedDomain,nbSharedDomains,avgPostsPerAcitveWeek,avgPostsPerWeek,page_verified,page_created_month,topAdminCountries,languageCodes,pageCategories,accountNames,nb_posts,topAdminCountry,languageCode,pageCategory,nbTopAdminCountries,nbLanguageCodes,nbPageCategories,nbAccountNames,adLib_NameChanged,adLib_AdminCountries,adLib_Organization,adLib_Address,adLib_TopAdminCountry,adLib_OrgCountry,adLib_nbAdmins,adLib_nbAdminCountries,is_dedicated,actual_pageid,page_id_2,profile_owner,first_detected_date,detected_2020,detected_2022,niche,history_file,first_detected_date_2020,first_detected_date_2022,number_of_times_detected_2020,number_of_times_detected_2022
5257,8108,8108,8108,8108,8108,8109,105759908674,Algemeiner,2017-06-19,2022-06-20,0.0,116275.0,NEWS_SITE,33494,3116208.0,11893.923664,10555.0,262,True,True,en,US,99.599928,0.185108,0.214964,algemeiner.com | chabad.org,algemeiner.com,2,127.839695,128.329502,1,2009-07,['US'],['en'],['NEWS_SITE'],['Algemeiner'],317,US,en,NEWS_SITE,1,1,1,1,0.0,"United States (7), Mexico (1), Serbia (1)",NaN,NaN,US,NaN,9.0,3.0,False,NaN,-1,-1,2020-10-01,True,True,False,../data/pages_posts_history_minhkha/1057599086...,2020-10-01,2022-06-02,30,28
7615,11673,11673,11673,11673,11673,11676,172614870130,The Pappas Post,2017-07-03,2022-06-13,0.0,151769.0,NEWS_SITE,5057,1493615.0,5834.433594,5021.5,256,True,True,en,US,87.818865,8.404192,3.776943,pappaspost.com | amazon.com | indiegogo.com | ...,pappaspost.com,5,19.753906,19.600775,0,2009-04,['US'],['en'],['NEWS_SITE'],['The Pappas Post'],23,US,en,NEWS_SITE,1,1,1,1,1.0,United States (4),NaN,NaN,US,NaN,4.0,1.0,False,NaN,-1,-1,2020-10-01,True,True,False,../data/pages_posts_history_minhkha/1726148701...,2020-10-01,2022-06-17,17,1
12383,19049,19049,19049,19049,19049,19053,321390109712,Jewish Journal,2017-06-19,2022-06-20,0.0,28637.0,MEDIA_NEWS_COMPANY,9323,342981.0,1334.556420,901.0,257,True,True,en,US,95.1517751796632,2.3704816046337016,2.4777432157031,jewishjournal.com | surveymonkey.com | eventbr...,jewishjournal.com,3,36.276265,35.720307,1,2010-02,['US'],['en'],['MEDIA_NEWS_COMPANY'],['Jewish Journal'],48,US,en,MEDIA_NEWS_COMPANY,1,1,1,1,1.0,United States (9),TRIBE MEDIA CORP.,"Los Angeles, Los Angeles County, California, U...",US,US,9.0,1.0,False,NaN,-1,-1,2020-10-01,True,True,False,../data/pages_posts_history_minhkha/3213901097...,2020-10-01,2022-06-02,26,5


In [33]:
print("number of pages with geographical content", len(PAGES_WITH_GEOGRAPHICAL_CONTENT), "fraction:", len(PAGES_WITH_GEOGRAPHICAL_CONTENT)/US_PAGES.shape[0])




number of pages with geographical content 9452 fraction: 0.5708074159067577


In [37]:
0.37 * len(US_PAGES)

6126.83